In [1]:
import ipywidgets as widgets
import numpy as np
import keras

from IPython import display as nb

2024-10-29 11:53:20.881901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 11:53:21.002389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 11:53:21.032689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 11:53:21.246527: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 11:53:22.992288: W tensorflow/compiler/tf2

In [2]:
packed_model = keras.models.load_model("../model/2024-10-29_00-09-51.keras")
packed_model.layers

I0000 00:00:1730177606.527057     746 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730177606.895274     746 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730177606.895628     746 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730177606.911490     746 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730177606.911584     746 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

[<WordPieceTokenizer name=word_piece_tokenizer_1, built=False>,
 <StringLookup name=string_lookup_1, built=False>,
 <Sequential name=sequential_2, built=True>]

In [3]:
tokenizer = packed_model.layers[0]
label_lookup = packed_model.layers[1]

model = packed_model.layers[2]
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 64, 100)        │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        34,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │            28 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 172,400 (673.44 KB)

 Trainable params: 57,466 (224.48 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 114,934 (448.96 KB)

In [5]:
w_text = widgets.Text(placeholder="Task")
w_text.continuous_update = False
nb.display(w_text)

def on_submit(value):
    nb.clear_output()
    nb.display(w_text)

    task = value["new"].strip()
    print(f"Input: {task}")

    task_vec = tokenizer(task).numpy()
    print(f"Vectorized input: {tokenizer.detokenize(task_vec)}")
    print(f"{task_vec}\n")
    task_vec = np.reshape(task_vec, (1, len(task_vec)))

    prediction = model.predict(task_vec)[0]
    print(f"Prediction vector: {prediction}")

    label_vocab = label_lookup.get_vocabulary()
    for label, probability in zip(label_vocab, prediction):
        percentage = probability * 100
        print(f"\t{label}: {percentage:.5f}%")

    best = label_vocab[np.argmax(prediction)]
    print(f"Prediction: {best}")

w_text.observe(on_submit, "value")


Text(value='Browse and buy a birthday present for her mom, whose birthday is next week. ', continuous_update=F…

Input: Browse and buy a birthday present for her mom, whose birthday is next week.
Vectorized input: browse and buy a birthday present for her mom [UNK] whose birthday is next week [UNK] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[130 201   8 225  50   7   8 222  71  45  73  52  75  22  71  68 123  36
  14  53  19 194   1  29  73 193 128   8 222  71  45  73  52  75  15  33
 177  29 183   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction vector: [9.9998546e-01 7.2630928e-06 5.3004931e-07 6.7390465e-06]
	not-urgent important: 99.99855%
	not-urgent not-important: 0.00073%
	urgent important: 0.00005%
	urgent not-important: 0.00067%
Prediction: not-urgent important
